In [3]:
import pandas as pd
import os
import pandas as pd
import numpy as np
import pyeeg
import pickle
from tqdm import tqdm
import mne
import picard
import time
import sys
import logging
import statistics
import pywt
logging.getLogger("mne").setLevel(logging.WARNING)
#тут нужно указать путь в __init__.py из папки entropy

MODULE_PATH = "C:/Users/Никита/Desktop/entropy/__init__.py"

MODULE_NAME = "entropy"

import importlib
import sys

spec = importlib.util.spec_from_file_location(MODULE_NAME, MODULE_PATH)
module = importlib.util.module_from_spec(spec)
sys.modules[spec.name] = module 
spec.loader.exec_module(module)

from entropy import *

import warnings
warnings.filterwarnings('ignore')


In [4]:
#функции для извлечения фичей из сырого ээг - сигнала

def first_der(x):
    return np.mean(np.abs(x[1:]-x[0:-1]))

def second_der(x):
    return np.mean(np.abs(x[2:] - x[0:-2]))

def first_der_norm(x):
    return first_der(x/max(np.abs(x)))

def second_der_norm(x):
    return second_der(x/max(np.abs(x)))

def samp_entropy(X):
    return sample_entropy(X)

def spec_entropy(X):
    return spectral_entropy(X, 100)

def sing_entropy(X):
    return svd_entropy(X)

def petrosyan(X):
    return petrosian_fd(X)

def Hig(X):  
    return higuchi_fd(X)

def Katz(X):
    return katz_fd(X)

def power_4_8(X):
    ##X = X * CONST
    return pyeeg.bin_power(X, [4, 8], 128)[0][0]

def power_8_12(X):
    return pyeeg.bin_power(X, [8, 12], 128)[0][0]

def power_12_25(X):
    return pyeeg.bin_power(X, [12, 25], 128)[0][0]

def power_35_45(X):
    return pyeeg.bin_power(X, [35, 45], 128)[0][0]

def theta_part(theta, alpha, beta, gamma):
    return 
    
def alpha_part(theta, alpha, beta, gamma):
    #get part from power_X_Y
    return 
def beta_part(theta, alpha, beta, gamma):
    #get part from power_X_Y
    return 

def gamma_part(theta, alpha, beta, gamma):
    #get part from power_X_Y
    return 

def freq_delta(X, Fs=128):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    delta = frequency[L*1//Fs-1: L*4//Fs]
    return np.std(delta)

def freq_theta(X, Fs=128):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    theta = frequency[L*4//Fs-1: L*8//Fs]
    return np.std(theta)

def freq_alpha(X, Fs=128):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    alpha = frequency[L*5//Fs-1: L*13//Fs]
    return np.std(alpha)

def freq_beta(X, Fs=128):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    beta = frequency[L*13//Fs-1: L*30//Fs]
    return np.std(beta)

def freq_gamma(X, Fs=128):
    L = len(X)
    data_fft = np.fft.fft(X)
    frequency = abs(data_fft/L)
    frequency = frequency[: L//2+1]*2
    gamma = frequency[L*30//Fs-1: L*50//Fs]
    return np.std(gamma)

def wvlt_gamma(X):
    coeffs = pywt.wavedec(X, 'db1', level=5)
    cA2, cD2,cD3,cD4,cD5, cD6 = coeffs
    gamma = cD5
    beta = cD4
    alpha = cD3
    theta = cD2
    return sum(gamma**2)

def wvlt_beta(X):
    coeffs = pywt.wavedec(X, 'db1', level=5)
    cA2, cD2,cD3,cD4,cD5, cD6 = coeffs
    gamma = cD5
    beta = cD4
    alpha = cD3
    theta = cD2
    return sum(beta**2)
def wvlt_alpha(X):
    coeffs = pywt.wavedec(X, 'db1', level=5)
    cA2, cD2,cD3,cD4,cD5, cD6 = coeffs
    gamma = cD5
    beta = cD4
    alpha = cD3
    theta = cD2
    return sum(alpha**2)
def wvlt_theta(X):
    coeffs = pywt.wavedec(X, 'db1', level=5)
    cA2, cD2,cD3,cD4,cD5, cD6 = coeffs
    gamma = cD5
    beta = cD4
    alpha = cD3
    theta = cD2
    return sum(theta**2)



functions = [np.mean,
             #np.std,
             first_der,second_der,statistics.median,
             #first_der_norm, second_der_norm,perm_entropy,app_entropy, samp_entropy,
            #spec_entropy, sing_entropy, petrosyan, Hig, Katz,
             power_4_8, power_8_12, power_12_25, power_35_45,
            #freq_delta, freq_theta, freq_alpha,
             freq_beta, freq_gamma,
            wvlt_gamma,wvlt_beta,wvlt_alpha,wvlt_theta]
def remove_eye_activity_ICA(data):
    good_chanels = [2, 3, 4, 5, 8, 12, 14, 32, 30, 26, 22, 21, 20, 18]
    ch_names = ["AF3", "F3", "F7", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F8", "F4","AF4"]
    EOG_chanels = [33, 34, 35, 36]
    eeg = []
    
    for ch in good_chanels:
        eeg.append(np.array(data[ch]))

    for ch in EOG_chanels:
        eeg.append(np.array(data[ch]))
        
    eeg = np.array(eeg) * 10e-6              #in volts
    
    info = mne.create_info(ch_names, 128, ch_types=["eeg"] * 14 + ["eog"] * 4)
    raw = mne.io.RawArray(eeg, info)
    
    raw_tmp = raw.copy()
    raw_tmp.filter(1, None)
    
    ica = mne.preprocessing.ICA(method="picard",
                            fit_params={"extended": True},
                            random_state=1)
    ica.fit(raw_tmp)
    #TODO: WHAT ICA'S ?
    #ica.exclude = [1]
    
    raw_corrected = raw.copy()
    
    return raw_corrected.get_data()

def remove_eye_activity_regr(data):
    good_chanels = [2, 3, 4, 5, 8, 12, 14, 32, 30, 26, 22, 21, 20, 18]
    EOG_chanels = [33, 34, 35, 36]
    eeg = []
    for ch in good_chanels:
        eeg.append(np.array(data[ch]))

    for ch in EOG_chanels:
        eeg.append(np.array(data[ch]))
    eeg = np.array(eeg) * 10e-6
    eeg1 = eeg.copy()
    info = mne.create_info(18, 512, ch_types=["eeg"] * 14 + ["eog"] * 4)
    raw1 = mne.io.RawArray(eeg, info)
    raw2 = mne.io.RawArray(eeg1, info)
    bip = np.array([[-1, 1, 0, 0], [0, 0, -1, 1]])
    raw1_eog = bip @ raw1[14:, :][0]
    raw2_eog = bip @ raw2[14:, :][0]
    raw1_eeg = raw1[:14, :][0]
    raw2_eeg = raw2[:14, :][0]
    b = np.linalg.inv(raw1_eog @ raw1_eog.T) @ raw1_eog @ raw1_eeg.T
    eeg_corrected = (raw2_eeg.T - raw2_eog.T @ b).T
    raw3 = raw2.copy()
    raw3._data[:14, :] = eeg_corrected
    return np.array(raw3.to_data_frame().T[1:])

def create_df(data, lables,subject, cut=2500, train=True, n_segments=10):
    features_all = []
    m = len(data[0])
    #нужные каналы
    
    good_chanels = [2, 3, 4, 5, 8, 12, 14, 32, 30, 26, 22, 21, 20, 18]
    ch_names = ["AF3", "F3", "F7", "FC5", "T7", "P7", "O1", "O2", "P8", "T8", "FC6", "F8", "F4","AF4"]
    #бьем нашу ээг на сегменты и бежим по ним в цикле 
    
    for j in range(n_segments):
        start = cut + j*(m - cut)//n_segments
        finish = cut + (j + 1)*(m - cut)//n_segments
        features = dict()
        for k in range(14):
            for func in functions:
                features[f"channel_{ch_names[k]}_{func.__name__}"] = func(data[k][start:finish])
                if func == power_35_45:
                    theta = features[f"channel_{ch_names[k]}_{power_4_8}"]
                    alpha = features[f"channel_{ch_names[k]}_{power_8_12}"]
                    beta = features[f"channel_{ch_names[k]}_{power_12_25}"]
                    gamma = features[f"channel_{ch_names[k]}_{power_35_45}"]
                    features[f"channel_{ch_names[k]}_theta_part"] = theta_part(theta, alpha, beta, gamma)
                    features[f"channel_{ch_names[k]}_theta_part"] = alpha_part(theta, alpha, beta, gamma)
                    features[f"channel_{ch_names[k]}_theta_part"] = beta_part(theta, alpha, beta, gamma)
                    features[f"channel_{ch_names[k]}_theta_part"] = gamma_part(theta, alpha, beta, gamma)
        features['arousal'] = lables[0]
        features['valence'] = lables[1]
        features['dominance'] = lables[2]
        features['liking'] = lables[3]
        features['subject'] = subject
        features_all.append(features)
    return pd.json_normalize(features_all)


In [5]:
datasets = []

# бежим по дериктории, применяем извлечение фичей ко всем файлам и склеиваем все фичи в один датасет

for fn in tqdm(os.listdir('data_preprocessed_python/')):
    with open('data_preprocessed_python/'+fn, 'rb') as f:
        raw = pickle.loads(f.read(), encoding='latin1')
        video = 0
        while video < 40:
            label = raw['labels'][video]
            #print('Video:', video + 1,'file:', fn)
            datasets.append(create_df(raw['data'][video], label, fn.split('.')[0]))
            video += 1

data_all = pd.concat(datasets)

100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [03:21<00:00,  6.31s/it]


In [8]:
#wawelet_features = list(filter(lambda x: x.endswith('wwlt'), data_all.columns))
data_all = data_all.sample(frac=1)
data_all
print('Theta', np.mean(data_all['channel_O2_power_4_8']))
print('alpha', np.mean(data_all['channel_O2_power_8_12']))
print('beta', np.mean(data_all['channel_O2_power_12_25']))
print('gamma', np.mean(data_all['channel_O2_power_35_45']))
data_all

Theta 20353.715703268226
alpha 10085.153255025272
beta 16327.471143067049
gamma 14802.272910307574


,channel_AF3_mean,channel_AF3_first_der,channel_AF3_second_der,channel_AF3_median,channel_AF3_power_4_8,channel_AF3_power_8_12,channel_AF3_power_12_25,channel_AF3_power_35_45,channel_AF3_freq_beta,channel_AF3_freq_gamma,...,channel_AF4_freq_gamma,channel_AF4_wvlt_gamma,channel_AF4_wvlt_beta,channel_AF4_wvlt_alpha,channel_AF4_wvlt_theta,arousal,valence,dominance,liking,subject
5,0.207745,23.279305,33.294351,-0.310332,37998.128916,18118.615243,33436.894784,37501.510417,1.181874,1.283797,...,0.710402,29460.634755,19399.650935,16560.512327,5186.667844,6.82,2.03,6.05,1.73,s10
6,0.003868,5.805508,8.040788,0.075871,5860.709101,3338.559474,9138.815257,9457.995703,0.336976,0.338215,...,0.215708,4081.756409,4651.512692,5637.225768,934.243233,8.14,3.17,5.47,7.97,s03
5,-0.002186,4.965766,8.311841,0.042252,10061.079178,7319.873749,12029.209449,5935.331186,0.461271,0.238475,...,0.185915,11500.720186,21045.057272,12493.683889,2917.202214,2.36,6.19,2.45,5.71,s14
7,-0.701031,37.213373,50.646909,-2.994857,37717.266753,21129.867327,41062.802254,59769.268891,1.947739,2.353794,...,1.328652,155457.454692,111081.373318,187786.413189,115568.608794,5.01,2.95,4.04,5.09,s04
7,-0.064603,3.294744,5.263542,0.317700,4208.755992,3178.761867,9143.139396,3848.936039,0.352626,0.130041,...,0.136423,4040.935533,3482.019026,1757.129511,352.675467,2.85,1.00,9.00,1.12,s27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6,0.139618,13.772302,19.060173,1.035950,10887.536865,8614.680239,19314.160406,25862.051195,0.755357,0.862735,...,0.708184,43066.887387,40355.057399,30306.338995,5278.175148,2.51,5.65,2.90,1.97,s16
3,-0.049741,3.367318,5.627970,0.407542,4295.882182,3672.974530,9914.255938,3412.835545,0.366735,0.117461,...,0.122040,3689.550938,4169.566813,3349.424812,755.567168,9.00,8.14,9.00,8.12,s27
2,0.111818,5.384487,7.572448,0.363300,7264.880376,3375.916449,8902.256311,9538.051782,0.308023,0.310485,...,0.459198,27306.409689,40028.675874,73746.817316,21949.529895,7.60,6.78,7.86,7.47,s08
8,-0.692721,7.986763,13.935404,-0.040412,50414.324732,16068.307841,20032.109232,19436.531209,0.613658,0.326543,...,0.199586,19632.314447,28012.349044,64432.239689,13024.204247,5.06,4.97,3.91,3.12,s30


In [7]:

#for wfeature in wawelet_features:
#    for i in range(4):
#        data_all[wfeature + f"_{i}"] = data_all[wfeature].apply(lambda x: x[i])


In [7]:
#data_all.drop(wawelet_features, axis=1, inplace=True)

In [70]:
#перемешиваем фичи, для улучшения обучения
with open('features_and_lables.xlsx', 'rb') as f:
    data_all = pd.read_excel(f)


#data_all.to_excel('features_and_lables.xlsx', index=False)

10767    0.099180
662     -0.438890
4337    -0.201664
3864     0.022114
11829   -0.337139
           ...   
7458     0.186251
11545    0.041707
8149     0.046045
564      0.522625
4433    -0.100120
Name: channel_O2_mean, Length: 12800, dtype: float64

In [73]:
data_all['channel_FC5_mean']

10767    0.106035
662     -0.173328
4337    -0.119555
3864     0.005971
11829    0.531590
           ...   
7458     0.438328
11545   -0.088496
8149     0.037944
564      0.007205
4433    -0.120763
Name: channel_FC5_mean, Length: 12800, dtype: float64

In [9]:
# получаем X и Y для обучения

X = data_all.drop(['arousal', 'valence', 'dominance', 'liking', 'subject'], axis=1)
y = data_all['valence'].apply(lambda x: 0 if x <= 4 else 1 if x > 4 and x < 6 else 2)

In [10]:
#сама модель

from catboost import Pool, cv


cv_dataset = Pool(data=X,
                  label=y)

params = {"iterations": 10000,
          "depth": 6,
          "loss_function": "Logloss",
          "custom_metric": "Accuracy",
          "verbose": False}

scores = cv(cv_dataset,
            params,
            fold_count=3,
            plot="True")

MetricVisualizer(layout=Layout(align_self='stretch', height='500px'))

CatBoostError: c:/program files (x86)/go agent/pipelines/buildmaster/catboost.git/catboost/private/libs/target/target_converter.cpp:380: Target with classes must contain only 2 unique values for binary classification

In [11]:
from catboost import CatBoostClassifier

cat_arousal_model = CatBoostClassifier(iterations=10000, depth=6)
cat_dominance_model = CatBoostClassifier(iterations=10000, depth=6)
cat_valence_model = CatBoostClassifier(iterations=10000, depth=6)

In [12]:
def train_model(measure, data, model):
    X = data_all.drop(['arousal', 'valence', 'dominance', 'liking', 'subject'], axis=1)
    y = data_all[measure].apply(lambda x: 0 if x <= 4 else 1 if x > 4 and x < 6 else 2 )
    model.fit(X, y)
    return model

In [13]:
valence_model = train_model('valence', data_all, cat_valence_model)


Learning rate set to 0.012876
0:	learn: 1.0969223	total: 143ms	remaining: 23m 50s
1:	learn: 1.0951945	total: 217ms	remaining: 18m 5s
2:	learn: 1.0936192	total: 283ms	remaining: 15m 44s
3:	learn: 1.0921994	total: 355ms	remaining: 14m 47s
4:	learn: 1.0908312	total: 422ms	remaining: 14m 4s
5:	learn: 1.0889326	total: 492ms	remaining: 13m 39s
6:	learn: 1.0873993	total: 564ms	remaining: 13m 24s
7:	learn: 1.0859019	total: 627ms	remaining: 13m 2s
8:	learn: 1.0846717	total: 699ms	remaining: 12m 55s
9:	learn: 1.0831410	total: 764ms	remaining: 12m 42s
10:	learn: 1.0815934	total: 842ms	remaining: 12m 44s
11:	learn: 1.0803787	total: 919ms	remaining: 12m 44s
12:	learn: 1.0788571	total: 997ms	remaining: 12m 45s
13:	learn: 1.0777193	total: 1.07s	remaining: 12m 42s
14:	learn: 1.0765944	total: 1.14s	remaining: 12m 36s
15:	learn: 1.0754418	total: 1.2s	remaining: 12m 27s
16:	learn: 1.0743288	total: 1.27s	remaining: 12m 23s
17:	learn: 1.0731660	total: 1.35s	remaining: 12m 27s
18:	learn: 1.0720575	total: 1.

155:	learn: 0.9851705	total: 10.4s	remaining: 10m 56s
156:	learn: 0.9848410	total: 10.5s	remaining: 10m 56s
157:	learn: 0.9844012	total: 10.5s	remaining: 10m 56s
158:	learn: 0.9840120	total: 10.6s	remaining: 10m 55s
159:	learn: 0.9836554	total: 10.7s	remaining: 10m 55s
160:	learn: 0.9833820	total: 10.7s	remaining: 10m 55s
161:	learn: 0.9829269	total: 10.8s	remaining: 10m 55s
162:	learn: 0.9826775	total: 10.9s	remaining: 10m 55s
163:	learn: 0.9823897	total: 10.9s	remaining: 10m 55s
164:	learn: 0.9819663	total: 11s	remaining: 10m 56s
165:	learn: 0.9816251	total: 11.1s	remaining: 10m 56s
166:	learn: 0.9813185	total: 11.2s	remaining: 10m 57s
167:	learn: 0.9810740	total: 11.3s	remaining: 10m 58s
168:	learn: 0.9807282	total: 11.3s	remaining: 11m
169:	learn: 0.9803981	total: 11.4s	remaining: 11m 1s
170:	learn: 0.9801517	total: 11.5s	remaining: 11m 3s
171:	learn: 0.9797000	total: 11.6s	remaining: 11m 2s
172:	learn: 0.9794982	total: 11.7s	remaining: 11m 3s
173:	learn: 0.9791919	total: 11.7s	rem

311:	learn: 0.9399518	total: 21.3s	remaining: 11m 1s
312:	learn: 0.9397525	total: 21.4s	remaining: 11m 1s
313:	learn: 0.9395077	total: 21.4s	remaining: 11m
314:	learn: 0.9392963	total: 21.5s	remaining: 11m
315:	learn: 0.9391026	total: 21.6s	remaining: 11m
316:	learn: 0.9388141	total: 21.6s	remaining: 11m
317:	learn: 0.9386052	total: 21.7s	remaining: 11m
318:	learn: 0.9384389	total: 21.7s	remaining: 10m 59s
319:	learn: 0.9381471	total: 21.8s	remaining: 10m 59s
320:	learn: 0.9379426	total: 21.9s	remaining: 11m
321:	learn: 0.9377138	total: 22s	remaining: 11m
322:	learn: 0.9374845	total: 22s	remaining: 11m
323:	learn: 0.9372810	total: 22.1s	remaining: 11m
324:	learn: 0.9371810	total: 22.2s	remaining: 11m
325:	learn: 0.9370330	total: 22.3s	remaining: 11m
326:	learn: 0.9367068	total: 22.3s	remaining: 11m
327:	learn: 0.9364632	total: 22.4s	remaining: 11m 1s
328:	learn: 0.9362965	total: 22.5s	remaining: 11m 2s
329:	learn: 0.9361340	total: 22.6s	remaining: 11m 2s
330:	learn: 0.9358036	total: 22

467:	learn: 0.9075668	total: 32.5s	remaining: 11m 2s
468:	learn: 0.9073553	total: 32.6s	remaining: 11m 2s
469:	learn: 0.9071807	total: 32.7s	remaining: 11m 2s
470:	learn: 0.9070718	total: 32.7s	remaining: 11m 2s
471:	learn: 0.9068980	total: 32.8s	remaining: 11m 2s
472:	learn: 0.9066411	total: 32.9s	remaining: 11m 2s
473:	learn: 0.9064556	total: 33s	remaining: 11m 2s
474:	learn: 0.9063025	total: 33s	remaining: 11m 2s
475:	learn: 0.9061685	total: 33.1s	remaining: 11m 2s
476:	learn: 0.9060102	total: 33.2s	remaining: 11m 2s
477:	learn: 0.9058314	total: 33.2s	remaining: 11m 1s
478:	learn: 0.9056436	total: 33.3s	remaining: 11m 1s
479:	learn: 0.9054534	total: 33.4s	remaining: 11m 2s
480:	learn: 0.9052536	total: 33.5s	remaining: 11m 2s
481:	learn: 0.9050430	total: 33.5s	remaining: 11m 2s
482:	learn: 0.9048823	total: 33.6s	remaining: 11m 2s
483:	learn: 0.9047735	total: 33.7s	remaining: 11m 2s
484:	learn: 0.9045453	total: 33.8s	remaining: 11m 2s
485:	learn: 0.9042822	total: 33.8s	remaining: 11m 

623:	learn: 0.8813441	total: 43s	remaining: 10m 46s
624:	learn: 0.8811107	total: 43.1s	remaining: 10m 46s
625:	learn: 0.8810004	total: 43.2s	remaining: 10m 46s
626:	learn: 0.8808615	total: 43.2s	remaining: 10m 46s
627:	learn: 0.8807060	total: 43.3s	remaining: 10m 45s
628:	learn: 0.8805444	total: 43.4s	remaining: 10m 45s
629:	learn: 0.8802717	total: 43.4s	remaining: 10m 45s
630:	learn: 0.8801470	total: 43.5s	remaining: 10m 45s
631:	learn: 0.8799297	total: 43.6s	remaining: 10m 45s
632:	learn: 0.8797458	total: 43.6s	remaining: 10m 45s
633:	learn: 0.8795329	total: 43.7s	remaining: 10m 45s
634:	learn: 0.8792919	total: 43.8s	remaining: 10m 45s
635:	learn: 0.8790457	total: 43.8s	remaining: 10m 45s
636:	learn: 0.8788676	total: 43.9s	remaining: 10m 45s
637:	learn: 0.8787837	total: 44s	remaining: 10m 44s
638:	learn: 0.8785901	total: 44s	remaining: 10m 44s
639:	learn: 0.8784649	total: 44.1s	remaining: 10m 44s
640:	learn: 0.8782949	total: 44.1s	remaining: 10m 44s
641:	learn: 0.8781566	total: 44.2s

...

9381:	learn: 0.3304303	total: 11m 1s	remaining: 43.6s
9382:	learn: 0.3304031	total: 11m 1s	remaining: 43.5s
9383:	learn: 0.3303776	total: 11m 1s	remaining: 43.4s
9384:	learn: 0.3303605	total: 11m 1s	remaining: 43.4s
9385:	learn: 0.3303368	total: 11m 1s	remaining: 43.3s
9386:	learn: 0.3302964	total: 11m 1s	remaining: 43.2s
9387:	learn: 0.3302572	total: 11m 2s	remaining: 43.2s
9388:	learn: 0.3302416	total: 11m 2s	remaining: 43.1s
9389:	learn: 0.3302195	total: 11m 2s	remaining: 43s
9390:	learn: 0.3302066	total: 11m 2s	remaining: 42.9s
9391:	learn: 0.3301846	total: 11m 2s	remaining: 42.9s
9392:	learn: 0.3301463	total: 11m 2s	remaining: 42.8s
9393:	learn: 0.3300994	total: 11m 2s	remaining: 42.7s
9394:	learn: 0.3300777	total: 11m 2s	remaining: 42.7s
9395:	learn: 0.3300395	total: 11m 2s	remaining: 42.6s
9396:	learn: 0.3300231	total: 11m 2s	remaining: 42.5s
9397:	learn: 0.3299981	total: 11m 2s	remaining: 42.5s
9398:	learn: 0.3299578	total: 11m 2s	remaining: 42.4s
9399:	learn: 0.3299151	total: 

9534:	learn: 0.3261931	total: 11m 13s	remaining: 32.8s
9535:	learn: 0.3261720	total: 11m 13s	remaining: 32.8s
9536:	learn: 0.3261426	total: 11m 13s	remaining: 32.7s
9537:	learn: 0.3261007	total: 11m 13s	remaining: 32.6s
9538:	learn: 0.3260699	total: 11m 13s	remaining: 32.5s
9539:	learn: 0.3260426	total: 11m 13s	remaining: 32.5s
9540:	learn: 0.3260241	total: 11m 13s	remaining: 32.4s
9541:	learn: 0.3260006	total: 11m 13s	remaining: 32.3s
9542:	learn: 0.3259784	total: 11m 13s	remaining: 32.3s
9543:	learn: 0.3259582	total: 11m 13s	remaining: 32.2s
9544:	learn: 0.3259330	total: 11m 13s	remaining: 32.1s
9545:	learn: 0.3258971	total: 11m 13s	remaining: 32s
9546:	learn: 0.3258771	total: 11m 13s	remaining: 32s
9547:	learn: 0.3258252	total: 11m 13s	remaining: 31.9s
9548:	learn: 0.3257872	total: 11m 14s	remaining: 31.8s
9549:	learn: 0.3257468	total: 11m 14s	remaining: 31.8s
9550:	learn: 0.3257283	total: 11m 14s	remaining: 31.7s
9551:	learn: 0.3257063	total: 11m 14s	remaining: 31.6s
9552:	learn: 0

9685:	learn: 0.3219428	total: 11m 25s	remaining: 22.2s
9686:	learn: 0.3219024	total: 11m 25s	remaining: 22.1s
9687:	learn: 0.3218767	total: 11m 25s	remaining: 22.1s
9688:	learn: 0.3218584	total: 11m 25s	remaining: 22s
9689:	learn: 0.3218443	total: 11m 25s	remaining: 21.9s
9690:	learn: 0.3218328	total: 11m 25s	remaining: 21.9s
9691:	learn: 0.3218175	total: 11m 25s	remaining: 21.8s
9692:	learn: 0.3217903	total: 11m 25s	remaining: 21.7s
9693:	learn: 0.3217693	total: 11m 25s	remaining: 21.6s
9694:	learn: 0.3217441	total: 11m 25s	remaining: 21.6s
9695:	learn: 0.3217139	total: 11m 26s	remaining: 21.5s
9696:	learn: 0.3216714	total: 11m 26s	remaining: 21.4s
9697:	learn: 0.3216290	total: 11m 26s	remaining: 21.4s
9698:	learn: 0.3215799	total: 11m 26s	remaining: 21.3s
9699:	learn: 0.3215572	total: 11m 26s	remaining: 21.2s
9700:	learn: 0.3215217	total: 11m 26s	remaining: 21.2s
9701:	learn: 0.3214995	total: 11m 26s	remaining: 21.1s
9702:	learn: 0.3214832	total: 11m 26s	remaining: 21s
9703:	learn: 0

9837:	learn: 0.3177827	total: 11m 38s	remaining: 11.5s
9838:	learn: 0.3177531	total: 11m 38s	remaining: 11.4s
9839:	learn: 0.3177347	total: 11m 38s	remaining: 11.4s
9840:	learn: 0.3177123	total: 11m 38s	remaining: 11.3s
9841:	learn: 0.3176940	total: 11m 38s	remaining: 11.2s
9842:	learn: 0.3176789	total: 11m 38s	remaining: 11.1s
9843:	learn: 0.3176574	total: 11m 39s	remaining: 11.1s
9844:	learn: 0.3176232	total: 11m 39s	remaining: 11s
9845:	learn: 0.3175835	total: 11m 39s	remaining: 10.9s
9846:	learn: 0.3175703	total: 11m 39s	remaining: 10.9s
9847:	learn: 0.3175406	total: 11m 39s	remaining: 10.8s
9848:	learn: 0.3175052	total: 11m 39s	remaining: 10.7s
9849:	learn: 0.3174711	total: 11m 39s	remaining: 10.7s
9850:	learn: 0.3174256	total: 11m 39s	remaining: 10.6s
9851:	learn: 0.3174125	total: 11m 39s	remaining: 10.5s
9852:	learn: 0.3173986	total: 11m 39s	remaining: 10.4s
9853:	learn: 0.3173797	total: 11m 39s	remaining: 10.4s
9854:	learn: 0.3173583	total: 11m 39s	remaining: 10.3s
9855:	learn:

9988:	learn: 0.3139620	total: 11m 52s	remaining: 784ms
9989:	learn: 0.3139370	total: 11m 52s	remaining: 713ms
9990:	learn: 0.3139194	total: 11m 52s	remaining: 642ms
9991:	learn: 0.3138904	total: 11m 52s	remaining: 570ms
9992:	learn: 0.3138564	total: 11m 52s	remaining: 499ms
9993:	learn: 0.3138308	total: 11m 52s	remaining: 428ms
9994:	learn: 0.3137985	total: 11m 52s	remaining: 357ms
9995:	learn: 0.3137893	total: 11m 52s	remaining: 285ms
9996:	learn: 0.3137573	total: 11m 52s	remaining: 214ms
9997:	learn: 0.3137427	total: 11m 52s	remaining: 143ms
9998:	learn: 0.3137205	total: 11m 53s	remaining: 71.3ms
9999:	learn: 0.3137154	total: 11m 53s	remaining: 0us


In [14]:
valence_model.score(X, y)

0.979765625

In [17]:

#model_arousal = ''
#with open('valence.pkl', 'rb') as f:
#    model_arousal = pkl.load(f)
headers = data_all.columns.values.tolist()
arr = []
for i in range(len(valence_model.feature_importances_)):
    arr.append((valence_model.feature_importances_[i], headers[i]))
arr.sort(reverse=True)
arr

[(2.584549242772899, 'channel_O2_mean'),
 (2.2390635162660275, 'channel_P7_mean'),
 (1.8874166930865255, 'channel_FC6_mean'),
 (1.8712451414560483, 'channel_AF3_mean'),
 (1.8559602556677344, 'channel_FC5_mean'),
 (1.810458611802602, 'channel_P8_mean'),
 (1.7106802895070465, 'channel_T7_mean'),
 (1.6867173584984247, 'channel_F3_mean'),
 (1.676564162506058, 'channel_F7_mean'),
 (1.5637259689905767, 'channel_T8_mean'),
 (1.4839684827731332, 'channel_AF4_mean'),
 (1.4785261311603104, 'channel_O2_freq_gamma'),
 (1.4550643059694448, 'channel_F8_mean'),
 (1.4091251593578986, 'channel_F4_mean'),
 (1.1182160625359683, 'channel_O2_power_35_45'),
 (1.041529445937051, 'channel_O1_mean'),
 (0.9497413063550024, 'channel_FC6_freq_gamma'),
 (0.944951841124484, 'channel_AF3_freq_gamma'),
 (0.854164235726268, 'channel_T8_freq_gamma'),
 (0.8484184416591798, 'channel_AF3_power_35_45'),
 (0.7954695433359915, 'channel_FC5_freq_gamma'),
 (0.785952973231106, 'channel_P8_freq_gamma'),
 (0.7804280734554417, 'ch

In [39]:
model_arousal

In [ ]:
import pickle as pkl

pkl.dump(arousal_model, open("arousal_removed_eyes.pkl", "wb"))


In [ ]:
dominance_model = train_model('dominance', data_all, cat_dominance_model)
pkl.dump(dominance_model, open("dominance_removed_eyes.pkl", "wb"))

In [ ]:
valence_model = train_model('valence', data_all, cat_valence_model)
pkl.dump(valence_model, open("valence_removed_eyes.pkl", "wb"))

In [ ]:
with open('arousal.pkl', 'rb') as f:
    model_arousal = pkl.load(f)
with open('dominance.pkl', 'rb') as f:
    model_dominance = pkl.load(f)
with open('valence.pkl', 'rb') as f:
    model_valence = pkl.load(f)

In [ ]:
f = open('data_preprocessed_python/s01.dat', 'rb')
data = pickle.loads(f.read(), encoding='latin1')